In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
import plotly.plotly as py
import plotly.graph_objs as go
import os

plt.style.use('ggplot')

In [2]:
direc = os.getcwd()
direc

'/Users/Martijn/Documents/GitHub/PythonProjectQuarter1'

In [3]:
os.chdir('/Users/Martijn/Documents/GitHub/data')

In [4]:
#reading in the data
df_code = pd.read_csv("countriescodes.csv", sep = ",", encoding = 'latin-1')
df_wdi = pd.read_csv("WDI/WDIData.csv")

#creating a dataframe with only countries in Sub-Saharan Africa, the region Sub-Saharan Africa and the World.
df_code = df_code[df_code['sub-region'] == "Sub-Saharan Africa"]["alpha-3"]
region = pd.Series(["SSF","WLD"]) 
df_code = df_code.append(region)
df_wdi = df_wdi[df_wdi["Country Code"].isin(df_code)]

# Set the years from colums to rows
df = pd.melt(df_wdi,id_vars =["Country Name","Country Code","Indicator Name","Indicator Code"],
          var_name = "Years")
# Sort the values first by Country Name and then by Years
df = df.sort_values(["Country Name", "Years"])

In [8]:
def data_plot(indicator_code):
    df_und = df[df["Indicator Code"] == indicator_code]
    df_und = df_und.dropna()
    x = df_und[df_und["Country Name"] == 'World']["Years"]
    y1 = df_und[df_und["Country Name"] == 'World']["value"]
    y2 = df_und[df_und["Country Name"] == 'Sub-Saharan Africa']["value"]

    trace0 = go.Scatter(
        x = x,
        y = y1,
        mode = "lines + markers",
        name = "World"
        )
    trace1 = go.Scatter(
        x = x,
        y = y2,
        mode = "lines + markers",
        name = "Sub-Saharan Africa"
    )

    data = [trace0,trace1]

In [9]:
data_plot('SN.ITK.DEFC.ZS')

layout = dict(title = 'Undernourishment in Sub-Saharan Africa vs. the World',
              xaxis = dict(title = 'Years'),
              yaxis = dict(title = '% Undernourishment',
                          range = [0,30]),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='line-mode')

NameError: name 'data' is not defined

In [6]:
data_plot('AG.YLD.CREL.KG')

layout = dict(title = "Cereal Yield in Sub-Saharan Afriva vs. the World",
              xaxis = dict(title = 'Years'),
              yaxis = dict(title = "Cereal Yield (kg per Hectare)",
                          range = [0,4500]),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='line-mode')

NameError: name 'data' is not defined

In [11]:
df_map = df[df["Indicator Code"] == 'SN.ITK.DEFC.ZS']
df_map = df_map[df_map["Years"] == "2016"]

scl = [[0.0, 'rgb(255,127,80)'],[0.2, 'rgb(255,99,71)'],[0.4, 'rgb(255,0,0)'],\
            [0.6, 'rgb(220,20,60)'],[1, 'rgb(178,34,34)']]

df_map['text'] = df_map['Country Name']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_map['Country Code'],
        z = df_map['value'],
        locationmode = 'ISO-3',
        text = df_map['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 1
            ) ),
        colorbar = dict(
            title = "% Undernourishment")
        ) ]

layout = dict(
        title = 'Undernourishment in Sub-Saharan Africa (2016)',
        geo = dict(
            scope='africa',
            showlakes = True,
            lakecolor = 'rgb(135,206,235)',
            showland = True,
            landcolor = 'rgb(220,220,220)')
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )